In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [21]:
train = pd.read_csv('combined_dataset.csv',sep=',')
train.info()

test = pd.read_csv('enhanced_combined_dataset.csv',sep=',')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Technician ID          50 non-null     object
 1   Task ID                50 non-null     object
 2   Expertise Match        50 non-null     int64 
 3   Task Priority          50 non-null     int64 
 4   Task Duration          50 non-null     int64 
 5   Distance to Task (km)  50 non-null     int64 
 6   Task Completed         50 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 2.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Distance to Task (km)  200 non-null    int64  
 1   Priority               200 non-null    int64  
 2   Task Complexity        200 non-null    int64  
 3   Max Working Hours   

### Drop Unnecessary Columns & Rename Columns

In [22]:
train_drop = train.drop(columns=['Task Duration'])

test_drop = test.drop(columns=["Max Working Hours","Travel Time (minutes)","Overtime Cost ($)"])
test_drop = test_drop.rename(columns={'Priority': 'Task Priority', 'Penalty for Delay ($)': 'Penalty Cost'})

train_drop.info()
test_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Technician ID          50 non-null     object
 1   Task ID                50 non-null     object
 2   Expertise Match        50 non-null     int64 
 3   Task Priority          50 non-null     int64 
 4   Distance to Task (km)  50 non-null     int64 
 5   Task Completed         50 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 2.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Distance to Task (km)  200 non-null    int64
 1   Task Priority          200 non-null    int64
 2   Task Complexity        200 non-null    int64
 3   Equipment Required     200 non-null    int64
 4   Customer Rating        200 non-

### Calculate Penalty Cost in Test Data

In [23]:
penalty_ranges = {
    1: (10, 50),
    2: (20, 100),
    3: (30, 150),
    4: (40, 200),
    5: (50, 250)
}

def calculate_penalty(priority):
    low, high = penalty_ranges[priority]
    return np.random.randint(low, high)

train_drop['Penalty Cost'] = train_drop['Task Priority'].apply(calculate_penalty)

train_drop.head()
train_drop.info()

,Technician ID,Task ID,Expertise Match,Task Priority,Distance to Task (km),Task Completed,Penalty Cost
0,T005,J001,0,2,1,1,21
1,T006,J002,1,1,17,0,33
2,T005,J003,0,2,19,1,49
3,T009,J004,1,3,15,1,67
4,T003,J005,1,3,14,1,31


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Technician ID          50 non-null     object
 1   Task ID                50 non-null     object
 2   Expertise Match        50 non-null     int64 
 3   Task Priority          50 non-null     int64 
 4   Distance to Task (km)  50 non-null     int64 
 5   Task Completed         50 non-null     int64 
 6   Penalty Cost           50 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 2.9+ KB


### Derive New Columns

<p>
    logic:

expertise = 0 if 

	eqpt trained-1, eqpt required-1 and task comp-h, tech comp-l
	eqpt trained-0, eqpt required-0 and task comp-h, tech comp-l

	eqpt trained-0, eqpt required-1 and task comp-l, tech comp-h

	eqpt trained-1, eqpt required-0 and task comp-h, tech comp-l
	eqpt trained-0, eqpt required-1 and task comp-h, tech comp-l


expertise = 1 if 

	eqpt trained-1, eqpt required-1 and task comp-l, tech comp-h
    eqpt trained-1, eqpt required-0 and task comp-l, tech comp-h
	eqpt trained-0, eqpt required-0 and task comp-l, tech comp-h
</p>

In [24]:
merged_df = pd.merge(
    train_drop, test_drop, on=['Task Priority', 'Distance to Task (km)']
)
tech_merged_df = merged_df.sort_values(by=['Technician ID','Expertise Match'])
tech_merged_df = tech_merged_df[['Technician ID', 'Expertise Match', 'Equipment Required', 'Task Complexity']]
# tech_merged_df

In [25]:
np.random.seed(42)

# created an empty tech dictionary
techdata = [
    {'Technician ID': 'T001', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T002', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T003', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T004', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T005', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T006', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T007', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T008', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T009', 'Eqpt Trained': None, 'Tech Complexity': None},
    {'Technician ID': 'T010', 'Eqpt Trained': None, 'Tech Complexity': None}
]

tech_df = pd.DataFrame(techdata)


df = tech_merged_df[
    (tech_merged_df['Expertise Match'] == 1)
    & (tech_merged_df['Equipment Required'] == 1)
].copy()


df['Eqpt Trained'] = 1
df['Tech Complexity'] = df['Task Complexity'].apply(lambda x: np.random.randint(x, 11))

findtrainedtech = tech_df.merge(df, on='Technician ID', how='left', suffixes=('', '_new'))

findtrainedtech['Eqpt Trained'] = findtrainedtech['Eqpt Trained_new'].combine_first(findtrainedtech['Eqpt Trained'])
findtrainedtech['Tech Complexity'] = findtrainedtech['Tech Complexity_new'].combine_first(findtrainedtech['Tech Complexity'])

mask = findtrainedtech['Eqpt Trained'].isna()
findtrainedtech.loc[mask, 'Eqpt Trained'] = np.random.randint(0, 2, size=mask.sum())


findtrainedtech.drop(columns=['Eqpt Trained_new', 'Tech Complexity_new', 'Expertise Match' ,'Equipment Required','Task Complexity'], inplace=True)
findtrainedtech


,Technician ID,Eqpt Trained,Tech Complexity
0,T001,0,None
1,T002,0,None
2,T003,1.0,8.0
3,T004,1,None
4,T005,0,None
5,T006,0,None
6,T007,1.0,10.0
7,T008,0,None
8,T009,1.0,7.0
9,T009,1.0,7.0


In [26]:
np.random.seed(42)

findtechcomp = tech_merged_df.merge(findtrainedtech, on='Technician ID', how='left')
# findtechcomp

MAX_COMPLEXITY = 10  # define an upper limit

# --- STEP 1: Define helper function ---
def assign_tech_comp(row):
    """
    Assign Tech Complexity based on:
      - Eqpt Trained
      - Eqpt Required
      - Expertise Match
      - Task Complexity
    """
    # skip if Tech Complexity already filled
    if pd.notna(row['Tech Complexity']):
        return row['Tech Complexity']

    t_train = row['Eqpt Trained']
    t_req = row['Equipment Required']
    expertise = row['Expertise Match']
    task_comp = row['Task Complexity']

    # --- Logic from your description ---
    # EXPERTISE = 0 → task high, tech low  (usually underqualified)
    # EXPERTISE = 1 → task low, tech high (overqualified / expert)
    if expertise == 0:
        # Random number LOWER than task complexity (but >= 1)
        lower_bound = 1
        upper_bound = max(2, task_comp)
        return np.random.randint(lower_bound, upper_bound)

    elif expertise == 1:
        # Random number HIGHER than task complexity
        lower_bound = task_comp
        upper_bound = MAX_COMPLEXITY + 1
        return np.random.randint(lower_bound, upper_bound)

    # Default fallback if no match
    return task_comp


# --- STEP 2: Apply the function ---
findtechcomp['Tech Complexity'] = findtechcomp.apply(assign_tech_comp, axis=1)

# findtechcomp

avg_comp = findtechcomp.groupby('Technician ID')['Tech Complexity'].mean().reset_index().round()

tech_df = findtrainedtech.merge(avg_comp, on='Technician ID', suffixes=('', '_avg'))
tech_df = tech_df[['Technician ID', 'Eqpt Trained', 'Tech Complexity_avg']]
tech_df = tech_df.rename(columns={'Tech Complexity_avg': 'Tech Complexity'})
tech_df['Tech Complexity'] = tech_df['Tech Complexity'].astype(int)
tech_df['Eqpt Trained'] = tech_df['Eqpt Trained'].astype(int)
tech_df = tech_df.drop_duplicates()
# tech_df.to_csv('technicians.csv', index=False)

### Creating Historical Task Table

In [30]:
historical_merged_df = merged_df.drop(columns=['Penalty Cost_y'])
historical_tasks = historical_merged_df.rename(columns={'Penalty Cost_x': 'Penalty Cost', 'Distance to Task (km)':'Task Distance'})
# historical_tasks.info()
historical_tasks.head()

# historical_tasks.to_csv('historical_tasks.csv', index=False)

,Technician ID,Task ID,Expertise Match,Task Priority,Task Distance,Task Completed,Penalty Cost,Task Complexity,Equipment Required,Customer Rating
0,T009,J004,1,3,15,1,67,3,1,5
1,T003,J005,1,3,14,1,31,5,1,3
2,T010,J007,0,1,14,1,42,8,1,5
3,T010,J007,0,1,14,1,42,5,0,3
4,T004,J009,0,3,27,0,87,3,0,3


In [31]:
# make Task ID Unique
def make_unique_t_ids(df, column='Task ID'):
    counts = {}
    new_ids = []

    for task_id in df[column]:
        if task_id not in counts:
            counts[task_id] = 0
            new_ids.append(task_id)
        else:
            counts[task_id] += 1
            base_num = int(task_id[1:])  # Extract numeric part
            new_num = base_num + counts[task_id]
            new_id = f"J{new_num:03d}"
            # Ensure new_id is also unique
            while new_id in counts:
                counts[task_id] += 1
                new_num = base_num + counts[task_id]
                new_id = f"J{new_num:03d}"
            counts[new_id] = 0
            new_ids.append(new_id)
    df[column] = new_ids
    return df


In [33]:
historical_tasks = make_unique_t_ids(historical_tasks)
historical_tasks.to_csv('historical_tasks.csv', index=False)

### Creating Unassigned Task Table

In [35]:
unassigned_tasks = test_drop.rename(columns={'Distance to Task (km)':'Task Distance'})

start_num = 51
unassigned_tasks['Task ID'] = ['J' + str(i).zfill(3) for i in range(start_num, start_num + len(unassigned_tasks))]
# unassigned_tasks.info()

unique_unassigned_tasks = make_unique_t_ids(unassigned_tasks)
unique_unassigned_tasks

unassigned_tasks.to_csv('unassigned_tasks.csv', index=False)

,Task Distance,Task Priority,Task Complexity,Equipment Required,Customer Rating,Penalty Cost,Task ID
0,52,1,5,0,1,35,J051
1,93,5,7,0,4,220,J052
2,15,1,4,0,3,34,J053
3,72,3,6,0,2,105,J054
4,61,2,4,0,1,40,J055
...,...,...,...,...,...,...,...
195,77,4,6,0,1,104,J246
196,3,5,7,0,5,215,J247
197,70,1,10,0,1,18,J248
198,72,5,10,0,2,175,J249


### XGBoost ML -  Train Model

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# -----------------------------
# STEP 1. Load your data
# -----------------------------
# historical_tasks: Task ID, Technician ID, task_distance, task_priority, task_complexity,
#                   equipment_required, task_completed
# technician_data: Technician ID, eqpt_trained, tech_complexity
# unassigned_tasks: Task ID, task_distance, task_priority, task_complexity, equipment_required

# (Assume they are already loaded into pandas dataframes)

# -----------------------------
# STEP 2. Merge historical tasks with technician info
# -----------------------------
train = historical_tasks.merge(tech_df, on='Technician ID', how='left')

# -----------------------------
# STEP 3. Create derived matching features
# -----------------------------
# expertise_match = 1 if (equipment OK) AND (tech qualified)
train['Expertise Match'] = (
    ((train['Equipment Required'] == 0) | (train['Eqpt Trained'] == 1)) &
    (train['Tech Complexity'] >= train['Task Complexity'])
).astype(int)

# How much harder/easier the task is compared to tech skill
train['complexity_gap'] = train['Tech Complexity'] - train['Task Complexity']

# -----------------------------
# STEP 3.5: Historical success-rate features per tech
# -----------------------------
tech_stats = train.groupby('Technician ID').agg(
    completion_rate=('Task Completed', 'mean'),
    avg_customer_rating=('Customer Rating', 'mean'),
    avg_penalty_cost=('Penalty Cost', 'mean'),
    expertise_rate=('Expertise Match', 'mean'),
    num_tasks=('Task ID', 'count')
).reset_index()

# Merge stats back to train
train = train.merge(tech_stats, on='Technician ID', how='left')


# -----------------------------
# STEP 4. Define features and label
# -----------------------------
feature_cols = [
    'Technician ID', 'Task Distance', 'Task Priority',
    'Task Complexity', 'Equipment Required',
    'Tech Complexity', 'Eqpt Trained',
    'Expertise Match', 'complexity_gap',
    'completion_rate', 'avg_customer_rating', 'avg_penalty_cost', 'expertise_rate'
]
X = train[feature_cols]
y = train['Task Completed']

# -----------------------------
# STEP 5. One-hot encode Technician ID (so model learns tech-specific patterns)
# -----------------------------
preprocessor = ColumnTransformer(
    transformers=[('tech', OneHotEncoder(handle_unknown='ignore'), ['Technician ID'])],
    remainder='passthrough'  # keep numeric columns
)

# -----------------------------
# STEP 6. Split train/test
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# STEP 7. Build and train model
# -----------------------------
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb', XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric='logloss',
        random_state=42
    ))
])

model.fit(X_train, y_train)

# -----------------------------
# STEP 8. Evaluate
# -----------------------------
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_prob > 0.5).astype(int)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_prob):.3f}")

# -----------------------------
# STEP 9. Create TECH–TASK PAIRS for unassigned tasks
# -----------------------------
unassigned_pairs = unassigned_tasks.merge(tech_df, how='cross')

# Compute same features
unassigned_pairs['Expertise Match'] = (
    ((unassigned_pairs['Equipment Required'] == 0) | (unassigned_pairs['Eqpt Trained'] == 1)) &
    (unassigned_pairs['Tech Complexity'] >= unassigned_pairs['Task Complexity'])
).astype(int)

unassigned_pairs['complexity_gap'] = (
    unassigned_pairs['Tech Complexity'] - unassigned_pairs['Task Complexity']
)

# NEW Merge tech historical stats 
unassigned_pairs = unassigned_pairs.merge(tech_stats, on='Technician ID', how='left')

# -----------------------------
# STEP 10. Predict probabilities for each tech–task pair
# -----------------------------
X_new = unassigned_pairs[feature_cols]
unassigned_pairs['completion_prob'] = model.predict_proba(X_new)[:, 1]

# -----------------------------
# STEP 11. Pick the best tech for each task
# -----------------------------

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

assigned_softmax = []

for TaskID, group in unassigned_pairs.groupby('Task ID'):
    probs = softmax(group['completion_prob'].values)
    choice = np.random.choice(group['Technician ID'], p=probs)
    assigned_softmax.append((TaskID, choice, group.loc[group['Technician ID'] == choice, 'completion_prob'].values[0]))

assigned_softmax_df = pd.DataFrame(assigned_softmax, columns=['Task ID', 'Technician ID', 'completion_prob'])
print(assigned_softmax_df)
assigned_softmax_df.to_csv('assigned_tasks.csv', index=False)


# checking distribution of assignments among technicians
tech_best_assignments = (
    assigned_softmax_df
    .groupby('Technician ID', as_index=False)['Task ID']
    .count()
    .rename(columns={'Task ID': 'num_tasks_assigned'})
    .sort_values(by='num_tasks_assigned', ascending=False)
)

print(tech_best_assignments)

c:\Python312\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tech',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Technician ID'])])),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.8, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, ...))])

Accuracy: 0.833
ROC-AUC: 0.667
    Task ID Technician ID  completion_prob
0      J051          T010         0.961320
1      J052          T009         0.663287
2      J053          T002         0.991859
3      J054          T002         0.962603
4      J055          T002         0.986808
..      ...           ...              ...
195    J246          T002         0.962603
196    J247          T007         0.286907
197    J248          T001         0.296820
198    J249          T002         0.906893
199    J250          T006         0.748704

[200 rows x 3 columns]
  Technician ID  num_tasks_assigned
1          T002                  35
2          T003                  30
9          T010                  27
8          T009                  19
4          T005                  18
5          T006                  17
0          T001                  16
6          T007                  16
7          T008                  12
3          T004                  10


In [32]:
import joblib
joblib.dump(model, 'tech_assignment_model.pkl')

['tech_assignment_model.pkl']